<a href="https://colab.research.google.com/github/SeungHan0816/Colab_Practice/blob/main/Colab_Practice08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08-02 CNN으로 MNIST 분류하기

In [16]:
import torch
import torch.nn as nn

# PyTorch의 텐서 차원 순서 : 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

# 1채널 짜리를 입력받아서 32채널을 뽑아내며, 커널 사이즈는 3
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

# 32채널 짜리를 입력받아서 64채널을 뽑아내며, 커널 사이즈는 3
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

# 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정
# ceil_mode=False → 반올림 안 함 = 내림(floor) 사용
pool = nn.MaxPool2d(2)
print(pool)

# 지금까지는 선언만한 것, 아직 이들을 연결시키지는 않음.

print("-"*100)

# 이들을 연결시켜서 모델을 완성
# 입력을 첫번째 합성곱층을 통과, 합성곱층을 통과시킨 후의 텐서의 크기 확인
# 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존
out = conv1(inputs)
print(out.shape)

# 맥스풀링을 통과, 맥스풀링을 통과한 후의 텐서의 크기 확인
out = pool(out)
print(out.shape)

# 두번째 합성곱층을 통과 후 크기 확인
out = conv2(out)
print(out.shape)

# 맥스풀링 통과 후 크기 확인
out = pool(out)
print(out.shape)

print("-"*100)

# [1, 64, 7, 7]에서 차원 보기.
# out의 첫번째 차원이 몇인지 출력(배치 크기	1)
print(out.size(0))

# 두번째 차원이 몇인지 출력(채널 수 (64개 특징맵)	64)
print(out.size(1))

# 세번째 차원이 몇인지 출력(높이	7)
print(out.size(2))

# out의 네번째 차원을 출력(너비	7)
print(out.size(3))

print("-"*100)

# .view()를 사용하여 텐서를 펼치는 작업
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

print("-"*100)

# 전결합층(Fully-Connteced layer)를 통과, 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환
fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)


텐서의 크기 : torch.Size([1, 1, 28, 28])
Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
----------------------------------------------------------------------------------------------------
torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
----------------------------------------------------------------------------------------------------
1
64
7
7
----------------------------------------------------------------------------------------------------
torch.Size([1, 3136])
----------------------------------------------------------------------------------------------------
torch.Size([1, 10])


In [22]:
# CNN으로 MNIST 분류하기

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

# 만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 됨
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

# 데이터로더를 사용하여 배치 크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

# 클래스로 모델 설계
class CNN(torch.nn.Module):

  def __init__(self):
      super(CNN, self).__init__()
      # 첫번째층
      # ImgIn shape=(?, 28, 28, 1)
      #    Conv     -> (?, 28, 28, 32)
      #    Pool     -> (?, 14, 14, 32)
      self.layer1 = torch.nn.Sequential(
          torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

      # 두번째층
      # ImgIn shape=(?, 14, 14, 32)
      #    Conv      ->(?, 14, 14, 64)
      #    Pool      ->(?, 7, 7, 64)
      self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

      # 전결합층 7x7x64 inputs -> 10 outputs
      self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

      # 전결합층 한정으로 가중치 초기화
      torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
      out = self.fc(out)
      return out

# 모델 정의
model = CNN().to(device)

# 비용 함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 총 배치의 수 출력
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

# 총 배치의 수는 600, 배치 크기를 100 -> 훈련 데이터는 총 60,000개

# 모델 훈련
for epoch in range(training_epochs):
  avg_cost = 0  # 에포크당 평균 비용을 저장하기 위한 변수 초기화

  for X, Y in data_loader:  # 미니 배치 단위로 데이터를 꺼내옴. X는 입력 데이터, Y는 레이블
      # 이미지 데이터는 이미 (28x28) 크기를 가지므로, 별도의 reshape 필요 없음
      # 레이블 Y는 원-핫 인코딩이 아닌 정수형 클래스 레이블임
      X = X.to(device)  # 입력 데이터를 연산이 수행될 장치로 이동 (예: GPU)
      Y = Y.to(device)  # 레이블을 연산이 수행될 장치로 이동 (예: GPU)

      optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
      hypothesis = model(X)  # 모델을 통해 예측값(hypothesis)을 계산 (순전파 연산)
      cost = criterion(hypothesis, Y)  # 예측값과 실제값 Y 간의 손실(cost) 계산
      cost.backward()  # 역전파 연산을 통해 기울기 계산
      optimizer.step()  # 옵티마이저를 통해 파라미터 업데이트

      avg_cost += cost / total_batch  # 현재 배치의 비용을 전체 배치 수로 나누어 누적

  # 에포크가 끝날 때마다 평균 비용 출력
  # {:>4}	오른쪽 정렬, 폭 4칸
  # {:>.9}	오른쪽 정렬, 소수점 이하 9자리
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


총 배치의 수 : 600
[Epoch:    1] cost = 0.225560248
[Epoch:    2] cost = 0.0630551875
[Epoch:    3] cost = 0.0462681577
[Epoch:    4] cost = 0.0374339223
[Epoch:    5] cost = 0.0314037167
[Epoch:    6] cost = 0.0261414386
[Epoch:    7] cost = 0.0216277987
[Epoch:    8] cost = 0.0179725569
[Epoch:    9] cost = 0.0158094056
[Epoch:   10] cost = 0.0131764095
[Epoch:   11] cost = 0.00995210093
[Epoch:   12] cost = 0.00957405847
[Epoch:   13] cost = 0.00827191118
[Epoch:   14] cost = 0.00659538386
[Epoch:   15] cost = 0.00589284115


In [25]:
# 테스트 진행(이전 코드 무조건 실행하기!)

# 학습을 진행하지 않을 것이므로 torch.no_grad() 사용
# 역전파(gradient 계산)를 하지 않도록 설정하는 문법(테스트는 단순 예측만 하고 학습은 하지 않기에 사용)
with torch.no_grad():
  # 테스트 데이터를 모델에 입력하기 위한 준비
  # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동
  # len(mnist_test) → 배치 크기 = 10000
  # 테스트 이미지 전체를 꺼내서, CNN에 넣을 수 있도록 크기를 [10000, 1, 28, 28]으로 바꿈.

  # MNIST는 흑백이라 채널 = 1

  # .float() → uint8 형식(0~255)을 float32로 바꿔줌 (학습과 연산은 float이 기본)
  # .to(device) → GPU나 CPU로 텐서를 옮김
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)

  # 레이블(label)은 우리가 정답으로 알고 있는 실제 값, 즉 모델이 예측해야 할 목표 값
  # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동
  Y_test = mnist_test.targets.to(device)

  # 모델 예측 수행
  prediction = model(X_test)  # 테스트 데이터에 대해 모델이 예측한 결과값

  # 예측 결과와 실제 레이블 비교
  # # 예측된 클래스와 실제 레이블이 일치하는지 확인
  # == Y_test → 예측한 값과 실제 레이블이 같은지 비교 → True or False의 텐서(예:[True, True, False, ...])
  # torch.argmax(prediction, 1)의 1은 dim=1, 즉 "가로 방향(열 방향)"으로 최댓값을 찾는다는 뜻
  correct_prediction = torch.argmax(prediction, 1) == Y_test

  # 정확도 계산
  # True → 1.0, False → 0.0으로 바꾸기 위해 .float()
  # 정확도를 계산하기 위해 일치하는 예측의 평균을 구함(예: (1.0 + 0.0 + 1.0 + ...) / 총 개수)
  accuracy = correct_prediction.float().mean()

  # accuracy는 텐서 → .item()으로 파이썬 숫자로 바꿔서 출력
  print('Accuracy:', accuracy.item())  # 정확도를 출력

# with torch.no_grad(): 블록은 기울기 계산을 하지 않아 메모리 사용을 최적화
# 테스트 데이터는 view 메서드를 사용해 (배치 크기, 채널 수, 높이, 너비) 형태로 변환
# to(device)를 통해 GPU나 CPU로 전송

# 98%의 정확도를 얻음.


Accuracy: 0.9883999824523926


# 08-03 깊은 CNN으로 MNIST 분류하기

In [29]:
import torch
import torchvision.datasets as dasets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

class CNN(torch.nn.Module):

  def __init__(self):
      super(CNN, self).__init__()
      self.keep_prob = 0.5  # 드롭아웃 확률

      # L1: 첫 번째 합성곱층 (Conv Layer)
      # 입력 이미지 형태: (?, 28, 28, 1)
      # Conv2d: 출력 채널 32개, 커널 크기 3x3, 스트라이드 1, 패딩 1
      # ReLU: 활성화 함수
      # MaxPool2d: 커널 크기 2x2, 스트라이드 2로 다운샘플링 -> 출력 형태: (?, 14, 14, 32)
      self.layer1 = torch.nn.Sequential(
          torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

      # L2: 두 번째 합성곱층 (Conv Layer)
      # 입력 이미지 형태: (?, 14, 14, 32)
      # Conv2d: 출력 채널 64개, 커널 크기 3x3, 스트라이드 1, 패딩 1
      # ReLU: 활성화 함수
      # MaxPool2d: 커널 크기 2x2, 스트라이드 2로 다운샘플링 -> 출력 형태: (?, 7, 7, 64)
      self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

      # L3: 세 번째 합성곱층 (Conv Layer)
      # 입력 이미지 형태: (?, 7, 7, 64)
      # Conv2d: 출력 채널 128개, 커널 크기 3x3, 스트라이드 1, 패딩 1
      # ReLU: 활성화 함수
      # MaxPool2d: 커널 크기 2x2, 스트라이드 2, 패딩 1로 다운샘플링 -> 출력 형태: (?, 4, 4, 128)
      self.layer3 = torch.nn.Sequential(
          torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

      # L4: 첫 번째 선형층 (Fully Connected Layer)
      # 입력 노드 수: 4x4x128, 출력 노드 수: 625
      # ReLU: 활성화 함수
      # Dropout: 드롭아웃으로 과적합 방지, p=0.5(keep_prob = 0.5 → p=0.5 → 50% 확률로 뉴런을 꺼버림)

      # torch.nn.init: PyTorch의 초기화 함수 모음
      # 위에서 정의한 self.fc1의 가중치를 Xavier 균등분포로 초기화
      # self.fc1.weight: 전결합층 fc1의 가중치 텐서 (W 행렬)
      self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
      torch.nn.init.xavier_uniform_(self.fc1.weight)  # 가중치 초기화
      self.layer4 = torch.nn.Sequential(
          self.fc1,
          torch.nn.ReLU(),
          torch.nn.Dropout(p=1 - self.keep_prob)) # p=0.5

      # L5: 최종 선형층 (Fully Connected Layer)
      # 입력 노드 수: 625, 출력 노드 수: 10 (클래스 개수)
      self.fc2 = torch.nn.Linear(625, 10, bias=True)
      torch.nn.init.xavier_uniform_(self.fc2.weight)  # 가중치 초기화

  def forward(self, x):
      out = self.layer1(x)  # 첫 번째 합성곱층 통과
      out = self.layer2(out)  # 두 번째 합성곱층 통과
      out = self.layer3(out)  # 세 번째 합성곱층 통과
      # layer3에서 ?,2048로 한 뒤 layer4에 2048 입력 -> 625로 출력
      out = out.view(out.size(0), -1)  # 선형층에 입력하기 위해 텐서를 Flatten
      out = self.layer4(out)  # 첫 번째 선형층 통과
      out = self.fc2(out)  # 최종 선형층 통과
      return out  # 최종 출력 반환

# CNN 모델 정의
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
      # 이미지는 이미 28×28 크기이기 때문에 reshape할 필요 없다.
      # 레이블(label)은 one-hot 인코딩이 되어 있지 않다.
      X = X.to(device)
      Y = Y.to(device)

      optimizer.zero_grad()
      hypothesis = model(X)
      cost = criterion(hypothesis, Y)
      cost.backward()
      optimizer.step()

      avg_cost += cost / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

총 배치의 수 : 600
[Epoch:    1] cost = 0.191506103
[Epoch:    2] cost = 0.0545598269
[Epoch:    3] cost = 0.0383968614
[Epoch:    4] cost = 0.0298613328
[Epoch:    5] cost = 0.025512591
[Epoch:    6] cost = 0.0207889285
[Epoch:    7] cost = 0.0178523194
[Epoch:    8] cost = 0.0139354225
[Epoch:    9] cost = 0.0135297636
[Epoch:   10] cost = 0.0124471048
[Epoch:   11] cost = 0.0104616843
[Epoch:   12] cost = 0.00825766567
[Epoch:   13] cost = 0.0107027367
[Epoch:   14] cost = 0.00786881521
[Epoch:   15] cost = 0.00688065216


In [32]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

# 층을 더 깊게 쌓았는데 오히려 정확도가 줄음!
# 꼭 깊게 쌓는 것이 정확도를 올려주지는 않으며 효율적으로 쌓는 것도 중요하다!


Accuracy: 0.9861000180244446
